# Convolutional LSTM

In [ ]:
import torch.nn as nn
import torch
 
class ConvLSTMCell(nn.Module):
 
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """
 
        super(ConvLSTMCell, self).__init__()
 
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
 
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2 # 保证在传递过程中 （h,w）不变
        self.bias = bias
 
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim, # i门，f门，o门，g门放在一起计算，然后在split开
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)
 
    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state # 每个timestamp包含两个状态张量：h和c
 
        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis # 把输入张量与h状态张量沿通道维度串联
 
        combined_conv = self.conv(combined) # i门，f门，o门，g门放在一起计算，然后在split开
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
 
        c_next = f * c_cur + i * g  # c状态张量更新
        h_next = o * torch.tanh(c_next) # h状态张量更新
 
        return h_next, c_next # 输出当前timestamp的两个状态张量
 
    def init_hidden(self, batch_size, image_size):
        """
        初始状态张量初始化.第一个timestamp的状态张量0初始化
        :param batch_size:
        :param image_size:
        :return:
        """
        height, width = image_size
        init_h = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        init_c = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        return (init_h,init_c)
 
 
class ConvLSTM(nn.Module):
 
    """
    Parameters:参数介绍
        input_dim: Number of channels in input# 输入张量的通道数
        hidden_dim: Number of hidden channels # h,c两个状态张量的通道数，可以是一个列表
        kernel_size: Size of kernel in convolutions # 卷积核的尺寸，默认所有层的卷积核尺寸都是一样的,也可以设定不通lstm层的卷积核尺寸不同
        num_layers: Number of LSTM layers stacked on each other # 卷积层的层数，需要与len(hidden_dim)相等
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers # 是否返回所有lstm层的h状态
        Note: Will do same padding. # 相同的卷积核尺寸，相同的padding尺寸
    Input:输入介绍
        A tensor of size [B, T, C, H, W] or [T, B, C, H, W]# 需要是5维的
    Output:输出介绍
        返回的是两个列表：layer_output_list，last_state_list
        列表0：layer_output_list--单层列表，每个元素表示一层LSTM层的输出h状态,每个元素的size=[B,T,hidden_dim,H,W]
        列表1：last_state_list--双层列表，每个元素是一个二元列表[h,c],表示每一层的最后一个timestamp的输出状态[h,c],h.size=c.size = [B,hidden_dim,H,W]
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:使用示例
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """
 
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()
 
        self._check_kernel_size_consistency(kernel_size)
 
        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers) # 转为列表
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers) # 转为列表
        if not len(kernel_size) == len(hidden_dim) == num_layers: # 判断一致性
            raise ValueError('Inconsistent list length.')
 
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers
 
        cell_list = []
        for i in range(0, self.num_layers): # 多层LSTM设置
            # 当前LSTM层的输入维度
            # if i==0:
            #     cur_input_dim = self.input_dim
            # else:
            #     cur_input_dim = self.hidden_dim[i - 1]
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1] # 与上等价
            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))
 
        self.cell_list = nn.ModuleList(cell_list) # 把定义的多个LSTM层串联成网络模型
 
    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: 5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)
 
        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            b, _, _, h, w = input_tensor.size()  # 自动获取 b,h,w信息
            hidden_state = self._init_hidden(batch_size=b,image_size=(h, w))
 
        layer_output_list = []
        last_state_list = []
 
        seq_len = input_tensor.size(1) # 根据输入张量获取lstm的长度
        cur_layer_input = input_tensor
 
        for layer_idx in range(self.num_layers): # 逐层计算
 
            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len): # 逐个stamp计算
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],cur_state=[h, c])
                output_inner.append(h) # 第 layer_idx 层的第t个stamp的输出状态
 
            layer_output = torch.stack(output_inner, dim=1) # 第 layer_idx 层的第所有stamp的输出状态串联
            cur_layer_input = layer_output # 准备第layer_idx+1层的输入张量
 
            layer_output_list.append(layer_output) # 当前层的所有timestamp的h状态的串联
            last_state_list.append([h, c]) # 当前层的最后一个stamp的输出状态的[h,c]
 
        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]
 
        return layer_output_list, last_state_list
 
    def _init_hidden(self, batch_size, image_size):
        """
        所有lstm层的第一个timestamp的输入状态0初始化
        :param batch_size:
        :param image_size:
        :return:
        """
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states
 
    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        """
        检测输入的kernel_size是否符合要求，要求kernel_size的格式是list或tuple
        :param kernel_size:
        :return:
        """
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')
 
    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        """
        扩展到多层lstm情况
        :param param:
        :param num_layers:
        :return:
        """
        if not isinstance(param, list):
            param = [param] * num_layers
        return param
    
if __name__ == "__main__":
    data = torch.randn((5,6,3,30,30))
    model = ConvLSTM(input_dim=3,
                     hidden_dim=[64, 64, 128],
                     kernel_size=[(3, 3),(5,5),(7,7)],
                     num_layers=3,
                     batch_first=True,
                     bias = True,
                     return_all_layers = True)
    layer_output_list, last_state_list = model(data)
    
    last_layer_output = layer_output_list[-1]
    last_layer_last_h,last_layer_last_c = last_state_list[-1]
    
    print(last_layer_output[:,-1,...]==last_layer_last_h)

注意事项：要注意hidden_dim,kernel_size,num_layers三个参数在LSTM层上的一致。即：

len(kernel_size) == len(hidden_dim) == num_layers
如果hidden_dim=64，kernel_size = (3,3), num_layers=3: 会搭建一个3层的convLSTM网络，每一层的隐状态都是64通道，kernel_size=(3,3)

如果hidden_dim=[64,128,256]，kernel_size = (3,3), num_layers=3: 会搭建一个3层的convLSTM网络，各层的隐状态通道数分别是[64,128,256]，所有层的kernel_size==(3,3)

如果hidden_dim=[64,128,256]，kernel_size = [(3,3),(5,5),(7,7)], num_layers=3: 会搭建一个3层的convLSTM网络，各层的隐状态通道数分别是[64,128,256]，各层的kernel_size分别是[(3,3),(5,5),(7,7)]

https://blog.csdn.net/Strive_For_Future/article/details/115507864

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        # return score, tag_seq
        return tag_seq

In [3]:
# data
import numpy as np
import pandas as pd

PARTICIPANT_ID = 1 #4348

df_csv=pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_right.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
df_csv=df_csv[(df_csv['participant_ID']==PARTICIPANT_ID)] # take 1st participant data
df_csv=df_csv.drop(['participant_ID'],axis=1)

In [5]:
q_index_list = sorted(list(set(df_csv['Q_ID'])))

total_q_list = []
for q in q_index_list:
    df_q = df_csv[df_csv['Q_ID'] == q]
    text_list = list(df_q['text'])
    label_list = list(df_q['fixation1'])
    q_list = [text_list, label_list]
    total_q_list.append(q_list)

print(
    '\n',total_q_list[0], # first data with text and label
    '\n',total_q_list[0][0], # first data's text
    '\n',total_q_list[0][1], # first data's label
)



 [['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小'], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]] 
 ['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小'] 
 [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]


In [10]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 4 #5
HIDDEN_DIM = 4 #4

train_num = int(len(q_index_list)*0.8)
training_data = total_q_list[:train_num]
testing_data = total_q_list[train_num:]

# word_to_ix # unique word and its representing number
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {0: 0, 1: 1, START_TAG: 2, STOP_TAG: 3}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print('real label：'+ str(precheck_tags.tolist()))
    print('untrained label prediction：'+ str(model(precheck_sent)))

print('=============start BiLSTM+CRF model training=============')
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(200):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    if epoch%50 == 0:
        print(f'the{epoch}th epoch Loss:{loss[0]}')


output_path = 'ner_trained_model.cpt'
torch.save(model, output_path)
print('=============trained model saved=============\n\n')



print('=============load trained model and predict=============')
model_path = 'ner_trained_model.cpt'
trained_ner_model = torch.load(model_path)
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print('trained label prediction：' + str(model(precheck_sent)))

# time spent : 3min

real label：[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
untrained label prediction：[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=============start BiLSTM+CRF model training=============
the0th epoch Loss:20.18512725830078
the50th epoch Loss:12.962231636047363
the100th epoch Loss:8.370246887207031
the150th epoch Loss:4.185600280761719
=============trained model saved=============


=============load trained model and predict=============
trained label prediction：[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]


In [10]:
# # accuracy from scratch
# total_wrong = []
# acccuracy_score = []
# for q in range(47):
#     precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
#     predict = model(precheck_sent)
#     answer = training_data[q][1]
#     accuracy = sum([1 for i,j in zip(predict,answer) if i==j])/len(answer)
#     print(str(q+1) + " accuracy:", round(accuracy,2))
#     acccuracy_score.append(accuracy)
#     wrong_list = [i for i,j in zip(predict,answer) if i!=j] # wrong answer
#     total_wrong.append(wrong_list)

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# training predict score
total_result_list = []
for q in range(len(training_data)):
    precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = training_data[q][1]
    print(training_data[q][0])
    print(predict)
    print(answer)
    print("===============================")
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小']
[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
['外加', '電池', '將', '碳棒', '及', '銅棒', '放入', '硫酸銅', '𥚃', '將', '發生', '什麼', '反應', '電解', '反應', '電池', '反應', '氧化還原', '反應']
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
['以', '針筒', '吸取', '熱水', '並', '封住', '針頭', '將', '針頭', '活塞', '向', '外', '拉出', '裡面', '的', '水', '會', '有', '何', '現象', '水位', '不變', '水', '沸騰', '水', '向', '上', '四散', '各', '處']
[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]
['雌蕊', '的', '花柱', '內', '有', '多少', '個', '花粉管', '一', '個', '兩', '個', '很多', '個']
[0, 0, 1, 0, 1, 0, 0, 1

In [12]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.96,0.94,1.00,0.97
1,0.95,1.00,0.92,0.96
2,0.97,0.93,1.00,0.96
3,0.86,0.80,0.80,0.80
4,0.86,0.86,0.80,0.83
5,1.00,1.00,1.00,1.00
6,0.92,0.82,1.00,0.90
7,1.00,1.00,1.00,1.00
8,1.00,1.00,1.00,1.00
9,0.83,0.77,0.77,0.77


In [13]:
total_result_df.mean(axis=0)

accuracy     0.937143
precision    0.925000
recall       0.930714
f1score      0.926429
dtype: float64

---
testing data show

In [14]:
# for testing data
test_word_to_ix = {}
for sentence, tags in testing_data:
    for word in sentence:
        if word not in test_word_to_ix:
            test_word_to_ix[word] = len(test_word_to_ix)

In [15]:
total_result_list = []
for q in range(len(testing_data)):
    precheck_sent = prepare_sequence(testing_data[q][0], test_word_to_ix)
    predict = model(precheck_sent)
    answer = testing_data[q][1]
    print(testing_data[q][0])
    print(predict)
    print(answer)
    print("===============================")
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

['細長', '的', '玻璃管', '裝', '水', '1033.6', 'cm', '同時', '拿掉', '玻璃管', '上', '和', '下', '管', '蓋', '水', '會', '跑出來', '嗎', '會', '不會', '只', '會', '跑出', '一點點']
[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0]
['未成熟', '的', '香蕉', '和', '成熟', '香蕉', '加', '碘液', '後', '哪', '個', '會', '比較', '黑', '呢', '未成熟', '成熟', '一樣', '黑']
[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1]
['在', '抽氣', '密封罐', '內', '放置', '一', '手套', '將', '罐', '內', '氣體', '抽出', '後', '手套', '會', '有', '何', '變化', '手套', '縮小', '手套', '脹大', '手套', '不變']
[0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
['砝碼', '置於', '串聯', '和', '並聯', '的', '二', '個', '相同', '的', '彈簧', '下', '不計', '彈簧', '重量', 'A', 'B', 'C', 'D', '四', '個', '彈簧', '伸長量', '並聯', '之', '彈簧', 'C', 'D', '比較', '長', '串聯', '之', '彈簧', 'A', 'B', '比較',

In [16]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.44,0.00,0.00,0.00
1,0.42,0.33,0.57,0.42
2,0.42,0.31,0.62,0.42
3,0.52,0.08,0.09,0.09


In [17]:
total_result_df.mean(axis=0)

accuracy     0.4500
precision    0.1800
recall       0.3200
f1score      0.2325
dtype: float64

In [18]:
# # draw confusion matrix
# import matplotlib.pyplot as plt
# conf_matrix = confusion_matrix(y_true=answer, y_pred=predict)
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()